In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [2]:
# from langchain.agents import tool

# @tool
# def get_word_length(word: str) -> int:
#     """Returns the length of a word."""
#     return len(word)


# tools = [get_word_length]

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
tools = [TavilySearchResults(max_results=2)]

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [22]:
from langchain_community.tools.convert_to_openai import (
    format_tool_to_openai_function
)
llm_with_tools = llm.bind(
    functions=[
        format_tool_to_openai_function(t)
        for t in tools
    ],
)

In [15]:
llm_with_tools.kwargs['functions']

[{'name': 'tavily_search_results_json',
  'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
  'parameters': {'title': 'TavilyInput',
   'description': 'Input for the Tavily tool.',
   'type': 'object',
   'properties': {'query': {'title': 'Query',
     'description': 'search query to look up',
     'type': 'string'}},
   'required': ['query']}}]

In [11]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [12]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
res = agent_executor.invoke({"input": "2024年の京都のおすすめを検索して教えて"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2024年 京都 おすすめ'}`


[{'url': 'https://rtrp.jp/articles/7858/', 'content': '【2024年最新】京都おすすめ観光スポット43選！絶対に外さないスポットを厳選！ Neptune更新日：2024年01月04日 [[ article.favoriteCnt ]] お気に入り 13161631 View  のスポットから穴場スポットまでおすすめ観光スポットを43箇所おすすめしたいと思います。お寺や神社以外のスポットも掲載しています。京都を観光する時はぜひ参考にしてみてくださいね！  どの時期もおすすめの定番観光地。 かなり混雑していますが、いつ行っても楽しめます。 オススメは桜の時期！ 鹿苑寺\u3000金閣寺 鹿苑寺 金閣寺 京都府京都市北区金閣寺町1 金閣寺  世界中から人気の高い観光都市「京都」、修学旅行などで一度は訪れたことがあるんじゃないでしょうか？でも京都にはまだまだあなたの知らない魅力がたくさんあるんです！今回は京都のおすすめの観光スポットを定番のスポットから穴場スポットまでおすすめ観【2024年最新】京都おすすめ観光スポット43選! 絶対に外さないスポットを厳選! Neptune更新日：2024年01月04日 ツイート シェア はてぶ あとで 世界中から人気の高い観光都市「京都」、修学旅行などで一度は訪れたことがあるんじゃないでしょうか？ でも京都にはまだまだあなたの知らない魅力がたくさんあるんです! 今回は京都のおすすめの観光スポットを定番のスポットから穴場スポットまでおすすめ観光スポットを43箇所おすすめしたいと思います。 お寺や神社以外のスポットも掲載しています。 京都を観光する時はぜひ参考にしてみてくださいね! 当サイト内のおでかけ情報に関して 情報は記事掲載時点のものです。 施設によって営業時間の変更や休業などの可能性があります。'}, {'url': 'https://ja.kyoto.travel/specialopening/winter/sennen.php', 'content'

In [21]:
from IPython.display import Markdown
Markdown(res['output'])

2024年の京都のおすすめ情報を検索しました。以下の情報が見つかりました。

1. [【2024年最新】京都おすすめ観光スポット43選！絶対に外さないスポットを厳選！](https://rtrp.jp/articles/7858/)
   - お寺や神社以外のスポットも含め、43箇所のおすすめ観光スポットが紹介されています。桜の時期には特にオススメです。

2. [2024年3月17日(日) 美しい京都を未来につなぐ旅 -銘木師と紐解く上賀茂神社の木の秘密-](https://ja.kyoto.travel/specialopening/winter/sennen.php)
   - 2023年12月から2024年4月までの限定日に開催されるイベントです。上賀茂神社の木の秘密を銘木師と一緒に探索するツアーがあります。

詳細な情報はリンク先をご確認ください。